# Capítulo 14: Clasificación de Imágenes con Redes Neuronales Convolucionales Profundas

# Índice

- [Componentes básicos de las CNN](#componentes-básicos-de-las-cnn)
- [Comprensión de las CNN y las jerarquías de funciones](#comprensión-de-las-cnn-y-las-jerarquías-de-funciones)
- [Realización de Convoluciones Discretas](#realización-de-convoluciones-discretas)
- [Convoluciones Discretas en 1 Dimensión](#convoluciones-discretas-en-1-dimensión)
- [Relleno de entradas para controlar el tamaño de los mapas de características de salida](#relleno-de-entradas-para-controlar-el-tamaño-de-los-mapas-de-características-de-salida)
- [Determinación del tamaño de salida de la convolución](#determinación-del-tamaño-de-salida-de-la-convolución)
- [Convoluciones Discretas en 2 Dimensiones](#convoluciones-discretas-en-2-dimensiones)
- [Submuestreo o agrupación de capas](#submuestreo-o-agrupación-de-capas)
- [Trabajo con múltiples canales de entrada o de color](#trabajo-con-múltiples-canales-de-entrada-o-de-color)
- [Funciones de activación](#funciones-de-activación)
- [Funciones de pérdida para clasificación](#funciones-de-pérdida-para-clasificación)
- [Implementación de una CNN profunda usando PyTorch](#implementación-de-una-cnn-profunda-usando-pytorch)
- [Convertir Jupyter Notebook a Fichero Python](#convertir-jupyter-notebook-a-fichero-python)

## Componentes básicos de las CNN

Las redes neuronales convolucionales (CNN) son una familia de modelos que fueron originalmente inspirados en cómo funciona la corteza visual del cerebro humano cuando reconocemos objetos.

El desarrollo de las CNN se remonta a la década de 1990, cuando Yann LeCun y sus colegas propusieron una arquitectura NN novedosa para clasificar dígitos escritos a mano a partir de imágenes.

Debido al excelente desempeño de las CNN para tareas de clasificación de imágenes, este tipo particular de NN feedforward ganó mucha atención y condujo a enormes mejoras en el aprendizaje automático para visión computacional.

Varios años después, en 2019, Yann LeCun recibió el premio Turing (el premio más prestigioso en informática) por sus contribuciones al campo de inteligencia artificial (IA), junto con otros dos investigadores, Yoshua Bengio y Geoffrey Hinton.

## Comprensión de las CNN y las jerarquías de funciones

La extracción exitosa de características relevantes es clave para el rendimiento de cualquier algoritmo de aprendizaje automático y el aprendizaje automático tradicional. Los modelos se basan en características de entrada que pueden provenir de un experto en el dominio o se basan en la selección o extracción de características computacionales técnicas.

Ciertos tipos de NN, como las CNN, pueden aprender automáticamente características de datos sin procesar que son más útiles para una tarea particular. Por esta razón, es común considerar las capas CNN como características para extraer:

• Las primeras capas (las que están inmediatamente después de la capa de entrada) extraen características de bajo nivel a partir de datos sin procesar, y las capas posteriores (a menudo capas completamente conectadas, como en un perceptrón multicapa (MLP) utiliza estas características para predecir un objetivo continuo (etiqueta de valor o clase).

In [28]:
from IPython.display import Image, HTML
display(HTML("""
<div style="display: flex; justify-content: center;">
    <img src="./figures/14_01.png" format="png">
</div>
"""))

Ciertos tipos de NN multicapa y, en particular, las CNN profundas, construyen la llamada jerarquía de características combinando las características de bajo nivel en forma de capas para formar características de alto nivel.

Por ejemplo, si estamos tratando con imágenes, entonces el nivel bajo de características, como bordes y manchas, se extraen de capas anteriores, que se combinan para formar entidades de alto nivel.

Estas características de alto nivel pueden formar formas más complejas, como los contornos generales de objetos como edificios, gatos o perros.

In [29]:
from IPython.display import Image, HTML
display(HTML("""
<div style="display: flex; justify-content: center;">
    <img src="./figures/14_02.png" format="png">
</div>
"""))

Una CNN calcula mapas de características a partir de una imagen de entrada, donde cada elemento proviene de un parche local de píxeles en la imagen de entrada.

In [30]:
from IPython.display import Image, HTML
display(HTML("""
<div style="display: flex; justify-content: center;">
    <img src="./figures/14_03.png" format="png">
</div>
"""))

Las CNN funcionan muy bien en tareas relacionadas con imágenes y eso se debe en gran medida a dos ideas importantes:

• Conectividad escasa: Un único elemento en el mapa de características está conectado solo a una pequeña porción de píxeles (esto es muy diferente de conectarse a toda la imagen de entrada, como en el caso de los MLP).

• Compartir parámetros: Se utilizan los mismos pesos para diferentes parches de la imagen de entrada.

Como consecuencia directa de estas dos ideas, la sustitución de un sistema convencional, MLP completamente conectado con una capa de convolución disminuye sustancialmente la cantidad de pesos (parámetros) en la red, por lo que veremos una mejora en la capacidad de capturar características relevantes.

En el contexto de los datos de imágenes, tiene sentido suponer que las personas cercanas a los píxeles suelen ser más relevantes entre sí que los píxeles que están más alejados.

Normalmente, las CNN se componen de varias capas de submuestreo convolucionales que van seguidas de una o más capas conectadas al final. Las capas completamente conectadas son esencialmente un MLP.

Las capas de submuestreo, comúnmente conocidas como capas de agrupación, no tienen parámetros que se puedan aprender.
No hay pesos ni unidades de polarización en las capas de agrupación. Sin embargo, tanto la capa convolucional como la completamente conectada tienen pesos y sesgos que se optimizan durante el entrenamiento.

## Realización de Convoluciones Discretas

Para entender cómo funcionan las operaciones de convolución, comencemos con un convolución en una dimensión, que a veces se utiliza para trabajar con ciertos tipos de datos de secuencia, como texto.

Una convolución discreta (o simplemente convolución) es un elemento fundamental de operación en una CNN.

## Convoluciones Discretas en 1 Dimensión

Una convolución discreta para dos vectores, x y w, se denota por y = x * w, en el cual el vector x es nuestra entrada (a veces llamada señal) y w se llama filtro o núcleo.

Una convolución discreta se define matemáticamente de la siguiente manera:

In [31]:
from IPython.display import Image, HTML
display(HTML("""
<div style="display: flex; justify-content: center;">
    <img src="./figures/14_04.png" format="png">
</div>
"""))

El hecho de que la suma pase por índices de –∞ a +∞ parece extraño, principalmente porque en las aplicaciones de aprendizaje automático, siempre tratamos con vectores de características finitas.

Por ejemplo, si x tiene 10 características con índices 0, 1, 2, ..., 8, 9, entonces los índices –∞:-1 y 10:+∞ están fuera de los límites para x.

Por lo tanto, para calcular correctamente la suma que se muestra en el punto anterior fórmula, se supone que x y w están llenos de ceros.

Esto dará como resultado un vector de salida, y, que también tiene tamaño infinito, con muchos ceros también.

Dado que esto no es útil en situaciones prácticas, x se rellena sólo con un valor finito número de ceros.

Este proceso se llama relleno con ceros o simplemente relleno. Aquí, el número de ceros rellenados en cada lado se indica con p.

In [32]:
from IPython.display import Image, HTML
display(HTML("""
<div style="display: flex; justify-content: center;">
    <img src="./figures/14_05.png" format="png">
</div>
"""))

Supongamos que la entrada original, x, y el filtro, w, tienen n y m elementos, respectivamente, donde m ≤ n.

El vector acolchado, xp, tiene tamaño n + 2p. La fórmula práctica para calcular una convolución discreta cambiará a lo siguiente:

In [33]:
from IPython.display import Image, HTML
display(HTML("""
<div style="display: flex; justify-content: center;">
    <img src="./figures/14_06.png" format="png">
</div>
"""))

Ejemplo de que el tamaño del relleno es cero (p=0):

In [34]:
from IPython.display import Image, HTML
display(HTML("""
<div style="display: flex; justify-content: center;">
    <img src="./figures/14_07.png" format="png">
</div>
"""))

Observa que el filtro girado, wr, es desplazado en dos celdas cada vez que cambiamos.

Este cambio es otro hiperparámetro de una circunvolución, la zancada, s.

En este ejemplo, la zancada es dos, s = 2.

Tenga en cuenta que la zancada debe ser número positivo menor que el tamaño del vector de entrada.

In [35]:
from IPython.display import Image, HTML
display(HTML("""
<div style="display: flex; justify-content: center;">
    <img src="./figures/14_08.png" format="png">
</div>
"""))

## Relleno de entradas para controlar el tamaño de los mapas de características de salida

Hay tres modos de relleno que se utilizan comúnmente en práctica: plena, igual y válida.

• En modo completo, el parámetro de relleno, p, se establece en p = m – 1. Relleno completo aumenta las dimensiones de la salida; por lo tanto, rara vez se usa en arquitecturas CNN.

• Generalmente se utiliza el mismo modo de relleno para garantizar que el vector de salida tiene el mismo tamaño que el vector de entrada, x. En este caso, el parámetro de relleno, p, se calcula según el tamaño del filtro, junto con el requisito de que el tamaño de entrada y el tamaño de salida sean los mismos.

• Finalmente, calcular una convolución en modo válido se refiere al caso donde p = 0 (sin relleno).

El modo de relleno más utilizado en las CNN es el mismo relleno.

Una de sus ventajas sobre los otros modos de relleno es la misma. El relleno preserva el tamaño del vector, lo que facilita el diseño y una arquitectura de red más conveniente.

In [36]:
from IPython.display import Image, HTML
display(HTML("""
<div style="display: flex; justify-content: center;">
    <img src="./figures/14_09.png" format="png">
</div>
"""))

## Determinación del tamaño de salida de la convolución

Supongamos que el vector de entrada es de tamaño n y el filtro es de talla m.

Entonces, el tamaño de la salida resultante de y = x * w, con relleno p y zancada s, se determinarían de la siguiente manera:

In [37]:
from IPython.display import Image, HTML
display(HTML("""
<div style="display: flex; justify-content: center;">
    <img src="./figures/14_10.png" format="png">
</div>
"""))

In [38]:
from IPython.display import Image, HTML
display(HTML("""
<div style="display: flex; justify-content: center;">
    <img src="./figures/14_11.png" format="png">
</div>
"""))

## Convoluciones Discretas en 2 Dimensiones

Cuando trabajamos con entradas 2D, como una matriz, 𝑿𝑛1×𝑛2, y la matriz de filtro, 𝑾𝑚1×𝑚2, donde 𝑚1 ≤ 𝑛1 y 𝑚2 ≤ 𝑛2, entonces la matriz 𝒀=𝑿*𝑾 es el resultado de una convolución 2D entre 𝑿 y 𝑾. Esto es definido matemáticamente de la siguiente manera:

In [39]:
from IPython.display import Image, HTML
display(HTML("""
<div style="display: flex; justify-content: center;">
    <img src="./figures/14_12.png" format="png">
</div>
"""))

In [40]:
from IPython.display import Image, HTML
display(HTML("""
<div style="display: flex; justify-content: center;">
    <img src="./figures/14_13.png" format="png">
</div>
"""))

In [41]:
from IPython.display import Image, HTML
display(HTML("""
<div style="display: flex; justify-content: center;">
    <img src="./figures/14_14.png" format="png">
</div>
"""))

## Submuestreo o agrupación de capas

El submuestreo se aplica normalmente en dos formas de agrupación operaciones en CNN: Agrupación máxima y agrupación media (también conocida como agrupación promedio).

La capa de agrupación generalmente se denota por 𝑃𝑛1×𝑛2. Aquí el subíndice determina el tamaño del vecindario (el número de píxeles vecinos en cada dimensión) donde la operación máxima o media es realizada. A este tipo de vecindario nos referimos como tamaño de la agrupación.

In [42]:
from IPython.display import Image, HTML
display(HTML("""
<div style="display: flex; justify-content: center;">
    <img src="./figures/14_15.png" format="png">
</div>
"""))

La agrupación (max-pooling) introduce una invariancia local. Esto significa que pequeños cambios en EL vecindario local no cambia el resultado de la agrupación máxima.

Por lo tanto, ayuda a generar características que sean más resistentes al ruido en la entrada de datos.

En el siguiente ejemplo se muestra que la agrupación máxima de dos diferentes matrices de entrada, X1 y X2, dan como resultado la misma salida:

In [43]:
from IPython.display import Image, HTML
display(HTML("""
<div style="display: flex; justify-content: center;">
    <img src="./figures/14_16.png" format="png">
</div>
"""))

La agrupación disminuye el tamaño de las características, lo que resulta en una mayor capacidad de eficiencia computacional. Además, reducir el número de funciones puede reducir también el grado de sobreajuste.

Tradicionalmente, se supone que la agrupación no se superpone.

La agrupación generalmente se realiza en vecindarios que no se superponen, lo que se puede hacer estableciendo el parámetro de zancada igual al tamaño de la agrupación. Por ejemplo, una capa de agrupación que no se superpone, 𝑃𝑛1×𝑛2, requiere un paso parámetro s = (n1, n2).

Si bien la agrupación sigue siendo una parte esencial de muchas arquitecturas de CNN, también se han desarrollado varias arquitecturas CNN sin utilizar capas de agrupación.

En lugar de utilizar capas de agrupación para reducir el tamaño de la entidad, los investigadores usan capas convolucionales con un paso de 2.

## Trabajo con múltiples canales de entrada o de color

Una entrada a una capa convolucional puede contener una o más matrices 2D o matrices con dimensiones N1×N2 (por ejemplo, la altura de la imagen y ancho en píxeles).

Estas matrices N1×N2 se llaman canales.

Las implementaciones convencionales de capas convolucionales esperan una representación tensorial de rango 3 como entrada, por ejemplo, una representación tridimensional. matriz, 𝑿𝑁1×𝑁2×𝐶𝑖n, donde Cin es el número de canales de entrada.

Por ejemplo, consideremos imágenes como entrada a la primera capa de una CNN. Si la imagen está coloreada y utiliza el modo de color RGB, entonces Cin = 3 (para los canales de color rojo, verde y azul en RGB).

Sin embargo, si la imagen está en escala de grises, entonces tenemos Cin=1, porque solo hay un canal con los valores de intensidad de píxeles en escala de grises.

In [44]:
from IPython.display import Image, HTML
display(HTML("""
<div style="display: flex; justify-content: center;">
    <img src="./figures/14_17.png" format="png">
</div>
"""))

## Funciones de activación

Se tienen diferentes funciones de activación, como ReLU, sigmoide y tanh.

Algunas de estas funciones de activación, como ReLU, se utilizan principalmente en las capas intermedias (ocultas) de una NN para agregar no linealidades a nuestro modelo.

Otros, como sigmoide (para binario) y softmax (para multiclase), se agregan en la última capa (salida), lo que da como resultado probabilidades de membresía de clase como salida del modelo.

Si las activaciones sigmoidea o softmax no están incluidas en el capa de salida, entonces el modelo calculará los logits en lugar de las probabilidades de pertenencia a una clase.

## Funciones de pérdida para clasificación

Centrarse en los problemas de clasificación, dependiendo del tipo de problema (binario versus multiclase) y el tipo de salida (logits versus probabilidades), debemos elegir la función de pérdida apropiada para entrenar nuestro modelo.

• La entropía cruzada binaria es la función de pérdida para una clasificación binaria (con una sola unidad de salida).

• La entropía cruzada categórica es la función de pérdida para clasificación multiclase.

In [45]:
from IPython.display import Image, HTML
display(HTML("""
<div style="display: flex; justify-content: center;">
    <img src="./figures/14_18.png" format="png">
</div>
"""))

In [ ]:
import torch
import torch.nn as nn
import numpy as np

In [ ]:
####### Binary Cross-entropy
logits = torch.tensor([0.8])
probas = torch.sigmoid(logits)
target = torch.tensor([1.0])
bce_loss_fn = nn.BCELoss()
bce_logits_loss_fn = nn.BCEWithLogitsLoss()
print(f'BCE (w Probas): {bce_loss_fn(probas, target):.4f}')
print(f'BCE (w Logits): {bce_logits_loss_fn(logits, target):.4f}')

In [ ]:
####### Categorical Cross-entropy
logits = torch.tensor([[1.5, 0.8, 2.1]])
probas = torch.softmax(logits, dim=1)
target = torch.tensor([2])
cce_loss_fn = nn.NLLLoss()
cce_logits_loss_fn = nn.CrossEntropyLoss()
print(f'CCE (w Logits): {cce_logits_loss_fn(logits, target):.4f}')
print(f'CCE (w Probas): {cce_loss_fn(torch.log(probas), target):.4f}')

## Implementación de una CNN profunda usando PyTorch

In [46]:
from IPython.display import Image, HTML
display(HTML("""
<div style="display: flex; justify-content: center;">
    <img src="./figures/14_19.png" format="png">
</div>
"""))

In [ ]:
import torchvision
from torchvision import transforms
image_path = './'
transform = transforms.Compose([
    transforms.ToTensor()
])

mnist_dataset = torchvision.datasets.FashionMNIST(
    root=image_path, train=True,
    transform=transform, download=True
)
from torch.utils.data import Subset
mnist_valid_dataset = Subset(mnist_dataset, 
                             torch.arange(10000))
mnist_train_dataset = Subset(mnist_dataset, 
                             torch.arange(
                                 10000, len(mnist_dataset)
                            ))
mnist_test_dataset = torchvision.datasets.FashionMNIST(
    root=image_path, train=False,
    transform=transform, download=False
)

In [ ]:
from torch.utils.data import DataLoader
batch_size = 64
torch.manual_seed(1)
train_dl = DataLoader(mnist_train_dataset,
                      batch_size,
                      shuffle=True)

valid_dl = DataLoader(mnist_valid_dataset,
                      batch_size,
                      shuffle=False)

In [ ]:
model = nn.Sequential()
model.add_module(
    'conv1',
    nn.Conv2d(
        in_channels=1, out_channels=32,
        kernel_size=5, padding=2
    )
)
model.add_module('relu1', nn.ReLU())
model.add_module('pool1', nn.MaxPool2d(kernel_size=2))
model.add_module(
    'conv2',
    nn.Conv2d(
        in_channels=32, out_channels=64,
        kernel_size=5, padding=2
    )
)
model.add_module('relu2', nn.ReLU())
model.add_module('pool2', nn.MaxPool2d(kernel_size=2))

Al proporcionar la forma de entrada como una tupla (4, 1, 28, 28) (4 imágenes dentro del lote, 1 canal y tamaño de imagen 28×28), especificado en este ejemplo, calculamos la salida tenga una forma (4, 64, 7, 7), indicando mapas de características con 64 canales y un tamaño espacial de 7×7.

In [ ]:
x = torch.ones((4, 1, 28, 28))
model(x).shape

In [ ]:
model.add_module('flatten', nn.Flatten())
x = torch.ones((4, 1, 28, 28))
model(x).shape

In [ ]:
model.add_module('fc1', nn.Linear(3136, 1024))
model.add_module('relu3', nn.ReLU())
model.add_module('dropout', nn.Dropout(p=0.5))
model.add_module('fc2', nn.Linear(1024, 10))

In [ ]:
loss_fn = nn.CrossEntropyLoss()
optimizer = torch.optim.Adam(model.parameters(), lr=0.001)

In [ ]:
def train(model, num_epochs, train_dl, valid_dl):
    loss_hist_train = [0] * num_epochs
    accuracy_hist_train = [0] * num_epochs
    loss_hist_valid = [0] * num_epochs
    accuracy_hist_valid = [0] * num_epochs
    for epoch in range(num_epochs):
        model.train()
        for x_batch, y_batch in train_dl:
            x_batch = x_batch.cpu()
            y_batch = y_batch.cpu()
            pred = model(x_batch)
            loss = loss_fn(pred, y_batch)
            loss.backward()
            optimizer.step()
            optimizer.zero_grad()
            loss_hist_train[epoch] += loss.item()*y_batch.size(0)
            is_correct = (
                torch.argmax(pred, dim=1) == y_batch
            ).float()
            accuracy_hist_train[epoch] += is_correct.sum()
        loss_hist_train[epoch] /= len(train_dl.dataset)
        accuracy_hist_train[epoch] /= len(train_dl.dataset)

        model.eval()

        with torch.no_grad():
            for x_batch, y_batch in valid_dl:
                x_batch = x_batch.cpu()
                y_batch = y_batch.cpu()
                pred = model(x_batch)
                loss = loss_fn(pred, y_batch)
                loss_hist_valid[epoch] += \
                    loss.item()*y_batch.size(0)
                is_correct = (
                    torch.argmax(pred, dim=1) == y_batch
                ).float()
                accuracy_hist_valid[epoch] += is_correct.sum()
        loss_hist_valid[epoch] /= len(valid_dl.dataset)
        accuracy_hist_valid[epoch] /= len(valid_dl.dataset)
        
        print(f'Epoch {epoch+1} accuracy: '
              f'{accuracy_hist_train[epoch]:.4f} val_accuracy: '
              f'{accuracy_hist_valid[epoch]:.4f}')
        
    return loss_hist_train, loss_hist_valid, \
        accuracy_hist_train, accuracy_hist_valid

In [ ]:
torch.manual_seed(1)
num_epochs = 20
hist = train(model, num_epochs, train_dl, valid_dl)

In [ ]:
import matplotlib.pyplot as plt
x_arr = np.arange(len(hist[0])) + 1
fig = plt.figure(figsize=(12,4))
ax = fig.add_subplot(1, 2, 1)
ax.plot(x_arr, hist[0], '--o', label='Train loss')
ax.plot(x_arr, hist[1], '--c', label='Validation loss')
ax.legend(fontsize=15)
ax = fig.add_subplot(1, 2, 2)
ax.plot(x_arr, hist[2], '--o', label='Train acc.')
ax.plot(x_arr, hist[3], '--c', 
        label='Validation acc.')
ax.legend(fontsize=15)
ax.set_xlabel('Epoch', size=15)
ax.set_ylabel('Accuracy', size=15)
plt.show()

In [ ]:
pred = model(mnist_test_dataset.data.unsqueeze(1) / 255.)
is_correct = (
    torch.argmax(pred, dim=1) == mnist_test_dataset.targets
).float()
print(f'Test accuracy: {is_correct.mean():4f}')

## Convertir Jupyter Notebook a Fichero Python

In [ ]:
! python .convert_notebook_to_script.py --input ch14_notebook.ipynb --output ch14_notebook.py